In [1]:
import random
import math
import time
import csv
from sklearn.cluster import KMeans
import numpy as np

In [20]:
"""kmeans takes in a list of lists, which represents a list of data
points, an integer indicating the number of clusters, a threshold
value, a float, of when to stop running the algorithm, and another
integer, representing the number of dimensions of the data. kmeans
outputs a list of clusters, which are represented as lists of data,
which are represented as lists."""
def kmeans(filename, numclusters, threshold, dim):
    """Initializes all necessary placeholders and other variables,
    and starts timing. centers will contain the centers of clusters,
    bign is the number of data, numchanges is number of changes, and
    since the algorithm is going through the data the first time, the
    number of changes is the number of data. mindist is the minimum
    distance that a center is from a given point. member is the membership
    of the current data point being investigated, clusters is the final
    list of clusters that will be outputted."""
    data = getinput(filename)
    start = time.time()
    centers = []
    bign = len(data)
    numchanges = bign
    mindit = 0
    member = 0
    members = []
    clusters = []
    writ = []
    
    stride = len(data) // numclusters
    for i in range(0, len(data), stride):
        centers.append(data[i])
    
    """Runs a for loop to put the number of clusters lists into the
    final return list, and sets the first number of clusters data
    points as the initial centers."""
    for i in range(0, numclusters):
        clusters.append([])
    
    """intializes the members to be -1 so that the first run through
    will have a high number of changes so the while loop will run."""
    for i in range(0, bign):
        members.append(-1)
        
    updatecenter = 0
    distancetime = 0
    numdist = 0
    blah = 0
    
    """Runs a while loop as long as the proportion of the number of
    changes is larger than the user-input threshold value. Then resets
    clusters to be empty and adds in the appropriate lists that represent
    each cluster. Also resets the number of changes back to 0."""
    # while 1.0 * numchanges / bign > threshold:
    while numchanges == bign:
        blah += 1
        clusters = []
        for i in range(0, numclusters):
            clusters.append([])
        numchanges = 0
        """Runs a for loop through the number of data, and marks the
        distance between that data point and the first center as the
        minimum and sets its membership to 0, so that we have a value
        we can compare the rest to."""
        for i in range(0, bign):
            k = 0
            for j in range(0,numclusters):
                if (len(centers[j]) > 0):
                    k = j
                    break
            start1 = time.time()
            mindist = distance(data[i], centers[k], dim)
            numdist += 1
            member = k
            """Runs a for loop through the number of clusters/the
            number of centers, so that we can check the distance between
            the data point in question and every cluster center, and if
            a center is closer than the present closest one, reset the
            smallest distance and candidate membership."""
            for j in range(k+1, numclusters):
                if len(centers[j]) == 0:
                    continue
                newdist = distance(data[i], centers[j], dim)
                numdist += 1
                if newdist < mindist:
                    mindist = newdist
                    member = j
            """if the membership has changed, update the membership and
            add 1 to the number of changes, and then append the data point
            to the correct cluster."""
            if member != members[i]:
                numchanges += 1
                members[i] = member
            end1 = time.time()
            distancetime += end1 - start1
            clusters[member].append(data[i])
        """update the cluster centers once the clusters have been created."""
        start1 = time.time()
        centers = updateclustercenter(clusters, dim)
        end1 = time.time()
        updatecenter += end1 - start1
        if len(centers) != numclusters:
            print "Error!!!", len(centers)

    """Stops timing and prints the time the program took."""
    end = time.time()
    print "Run time: ", end - start
    
    output(members, centers, clusters)

In [3]:
def centergen(data, numcluster, dim):
    dataperclust = len(data) // numcluster
    temp = []
    for j in range(0, dim):
        temp = np.append(temp, 0)
    centers = np.array([temp1])
    temp = np.array([temp])
    
    n = 0
    for i in range(0, numcluster):
        for j in range(0, dim):
            temp[j] = 0
        for j in range(0, dataperclust):
            for k in range(0, dim):
                temp[k] += data[n][k]
            n += 1
        for j in range(0, dim):
            temp[j] /= dataperclust
        centers.append(centers, temp, axis=0)
    
    np.delete(centers, 0, 0)
        
    return centers

In [4]:
"""Distance takes two data points, data1 and data2, both of which are lists,
and an integer representing the number of dimensions, and calculates the
square of the distance betwee those two points."""
def distance(data1, data2, dim):
    """sets distance to 0, so we can start accumulating later."""
    distance = 0.0
    
    """for every dimension, square the difference between the two data
    points, and add it to distance to get the square of the distance."""
    for i in range(0, dim):
        distance += (data1[i] - data2[i]) ** 2
        
    return distance

In [18]:
"""updateclustercenter takes a list of list of lists, data, which is a 
list that contains lists, each of which represents one cluster, and in
those sub-lists are lists representing data. updateclustercenter also
takes the dimensions of the data as an input, an integer. Then it returns
a new list of centers that are the average of the data in each cluster."""
def updateclustercenter(clusters, dim):
    """Initialize the centers list to be empty, and the average to be 0."""
    temp = []
    centers = []
    avg = 0
    
    """Runs a for loop through the number of clusters, and sets a temp
    variable, that will contain the new center, to be empty."""
    for i in range(0, len(clusters)):
        if len(clusters[i]) == 0:
            centers.append(centers, clusters[len(clusters) // 3], axis=0)
            continue
        temp = []
        """Runs a for loop through the number of dimensions and sets the
        average value to be zero at the beginning of calculating the
        average of each dimension."""
        for j in range(0, dim):
            avg = 0.0
            """Runs a for loop through the number of data points in a given
            cluster, then adds that data point's value at that dimension.
            Then appends the total divided by the number of data to the temp
            variable that will hold the new center."""
            for k in range(0, len(clusters[i])):
                avg += clusters[i][k][j]
            temp.append(avg / len(clusters[i]))
        """Appends the new center to the list of centers."""
        centers.append(temp)
    return centers

In [6]:
def getinput(filename):
    finaldata = []
    onedata = []
    temp = ''
    
    f = open(filename, 'r')
    initdata = f.readlines()
    
    for i in range(0, len(initdata)):
        j = 0
        while initdata[i][j] != '\r':
            if initdata[i][j] == ',':
                onedata.append(float(temp))
                temp = ''
            else:
                temp += initdata[i][j]
            j += 1
        onedata.append(float(temp))
        finaldata.append(onedata)
        onedata = []
        temp = ''
        
    return finaldata

In [7]:
def output(members, centers, data):
    writ = []
    writ2 = []
    j = 0
    temp = 0
    
    for i in range(0, len(members)):
        writ.append([members[i]])
        
    for i in range(0, len(centers)):
        writ2.append(centers[i].append(len(data[i])))
            
    with open("kmeanscenters.csv", 'wb') as resultFile:
        wr = csv.writer(resultFile)
        wr.writerows(centers)
    
    with open("kmeanslabels.csv",'wb') as resultFile:
        wr = csv.writer(resultFile)
        wr.writerows(writ)

In [21]:
kmeans('output.csv', 12, 0.01, 3)
start = time.time()
X = getinput('output.csv')
kmeans = KMeans(n_clusters=12, random_state=0).fit(X)
end = time.time()
print 'Time: ', end - start
print kmeans.labels_

Run time:  2.62849783897
Time:  1.44773697853
[3 6 9 ..., 9 7 2]
